In [1]:
pip install nltk spacy python-dateutil speechrecognition

   ---------------------------------------- 0.0/32.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/32.9 MB ? eta -:--:--
    --------------------------------------- 0.5/32.9 MB 1.9 MB/s eta 0:00:18
   - -------------------------------------- 1.0/32.9 MB 2.1 MB/s eta 0:00:16
   - -------------------------------------- 1.6/32.9 MB 2.3 MB/s eta 0:00:14
   -- ------------------------------------- 2.4/32.9 MB 2.6 MB/s eta 0:00:12
   --- ------------------------------------ 2.9/32.9 MB 2.8 MB/s eta 0:00:11
   ---- ----------------------------------- 3.7/32.9 MB 2.8 MB/s eta 0:00:11
   ----- ---------------------------------- 4.5/32.9 MB 2.9 MB/s eta 0:00:10
   ------ --------------------------------- 5.0/32.9 MB 3.0 MB/s eta 0:00:10
   ------- -------------------------------- 5.8/32.9 MB 3.1 MB/s eta 0:00:09
   ------- -------------------------------- 6.6/32.9 MB 3.1 MB/s eta 0:00:09
   -------- ------------------------------- 7.3/32.9 MB 3.1 MB/s eta 0:00:09
   --------- 

In [7]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 3.4 MB/s eta 0:00:04
     ---- ----------------------------------- 1.3/12.8 MB 3.4 MB/s eta 0:00:04
     ------ --------------------------------- 2.1/12.8 MB 3.4 MB/s eta 0:00:04
     -------- ------------------------------- 2.6/12.8 MB 3.3 MB/s eta 0:00:04
     ---------- ----------------------------- 3.4/12.8 MB 3.3 MB/s eta 0:00:03
     ------------ --------------------------- 3.9/12.8 MB 3.3 MB/s eta 0:00:03
     -------------- ------------------------- 4.7/12.8 MB 3.3 MB/s eta 0:00:03
     ---------------- ----------------------- 5.2/12.8 MB 3.4 MB/s eta 0:00:03
     ------------------ --------------------- 6.0/12.8 MB 3.4 MB/s eta 0:00:03
     --------------------- ------------------ 6.8/12.8 MB 3.3 MB/s eta 0:00:02
     ---------------------- ----------------- 7.3/12.8 MB 3.3 MB/s eta 0:00:02
     ------------------------- -------------- 8.1/12.8 MB 3

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\msi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\msi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
import nltk
import spacy
from datetime import datetime, timedelta
from dateutil import parser as date_parser
import shelve
import os

nlp = spacy.load("en_core_web_sm")


def extract_datetime(text):
    """Extracts date and time information from text, handling "tomorrow" and "today"."""
    text = text.lower()
    if "tomorrow" in text:
        date_time = datetime.now().date() + timedelta(days=1)
        time_str = text.replace("tomorrow", "")
        try:
            time_part = date_parser.parse(time_str, fuzzy=True).time()
            date_time = datetime.combine(date_time, time_part)
        except ValueError:
            date_time = datetime.combine(date_time, datetime.min.time())
        return date_time
    elif "today" in text:
        date_time = datetime.now().date()
        time_str = text.replace("today", "")
        try:
            time_part = date_parser.parse(time_str, fuzzy=True).time()
            date_time = datetime.combine(date_time, time_part)
        except ValueError:
            date_time = datetime.combine(date_time, datetime.min.time())
        return date_time
    else:
        try:
            return date_parser.parse(text, fuzzy=True)
        except ValueError:
            return None


def store_appointment(date_time, description):
    """Stores an appointment in the shelve database."""
    with shelve.open("my_calendar") as db:
        db[str(date_time)] = description


def get_appointments(date=None):
    """Retrieves appointments from the shelve database.
    If date is None, returns all appointments.
    """
    appointments = []
    with shelve.open("my_calendar") as db:
        for key in db:
            appointment_date = date_parser.parse(key)
            if date is None or appointment_date.date() == date.date():
                appointments.append((appointment_date, db[key]))
    return appointments


def clear_calendar():
    """Clears all appointments from the shelve database."""
    confirmation = input(
        "Are you sure you want to clear your entire calendar? (yes/no): "
    ).lower()
    if confirmation == "yes":
        try:
            db_name = "my_calendar"
            db_path = f"{db_name}.db"
            try:
                db.close()
            except:
                pass

            extensions = [".bak", ".dat", ".dir"]
            for ext in extensions:
                try:
                    os.remove(f"{db_name}{ext}")
                except FileNotFoundError:
                    pass 
            print("Calendar cleared.")
        except Exception as e:
            print(f"Error clearing calendar: {e}")
    else:
        print("Calendar clearing cancelled.")


def clear_appointment(text):
    """Clears a specific appointment from the shelve database based on user input."""
    try:
        date_time = extract_datetime(text)
        if date_time:
            with shelve.open("my_calendar") as db:
                key_to_delete = str(date_time)
                if key_to_delete in db:
                    del db[key_to_delete]
                    print(
                        f"Appointment on {date_time.strftime('%Y-%m-%d %H:%M')} cleared."
                    )
                else:
                    print("No such appointment found on that date and time.")
        else:
            print(
                "Could not determine a valid date or time in your request. Please be more specific."
            )
    except Exception as e:
        print(f"Error clearing appointment: {e}")


def process_command(command):
    """Processes the user command and performs actions."""

    print(f"Command entered: {command}")

    if not command:
        print("Please enter a command.")
        return True

    # --- Intent Recognition (Simple Keyword-Based) ---
    if "quit" in command or "exit" in command or "bye" in command:
        intent = "quit"
    elif (
        "appointment" in command
        or "schedule" in command
        or "meeting" in command
        or "remind" in command
    ):
        intent = "schedule_appointment"
    elif "workout" in command or "exercise" in command:
        intent = "add_workout"
    elif "calendar" in command or "events" in command:
        intent = "view_calendar"
    elif "clear" in command and "calendar" in command:
        intent = "clear_calendar"
    elif "clear" in command and "appointment" in command:
        intent = "clear_appointment"
    else:
        intent = "unknown"

    print(f"Intent recognized: {intent}")

    # --- Entity Extraction ---
    entities = extract_entities(command)
    date_time = extract_datetime(command)

    # --- Action ---
    if intent == "schedule_appointment":
        if date_time:
            description = entities.get(
                "EVENT", None
            )
            if not description:
                description = input(
                    "What is the purpose of this appointment? "
                )
                if not description:
                    description = "Meeting"
            store_appointment(date_time, description)
            print(f"Okay, scheduling appointment for {description} on {date_time}")
        else:
            print("Could not determine date or time. Please be more specific.")
    elif intent == "add_workout":
        print("Okay, adding workout.")
    elif intent == "view_calendar":
        appointments = get_appointments()
        if appointments:
            print("Your upcoming appointments:")
            for appointment_date, description in appointments:
                print(
                    f"- {appointment_date.strftime('%Y-%m-%d %H:%M')}: {description}"
                )
        else:
            print("You have no appointments scheduled.")
    elif intent == "clear_calendar":
        clear_calendar()
    elif intent == "clear_appointment":
        clear_appointment(command)
    elif intent == "quit":
        return False
    else:
        print("I'm sorry, I didn't understand that.")

    return True


def get_user_input():
    """Gets user input from the console."""
    command = input("Enter your command: ")
    return command


def extract_entities(text):
    """Extracts entities from the text using spaCy."""
    doc = nlp(text)
    entities = {}
    for ent in doc.ents:
        entities[ent.label_] = ent.text
    return entities


if __name__ == "__main__":
    print("Hello! How can I help you today?")
    continue_running = True
    while continue_running:
        command = get_user_input()
        continue_running = process_command(command)

    print("Assistant shutting down. Goodbye for now!")